# Low Dimensional Connectome Dynamics

In [1]:
import bct
import copy
import matplotlib.cm as cmx
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pickle
import utils

import warnings; warnings.simplefilter('ignore')

from atlases import DesikanAtlas
from hmmlearn import hmm
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from tqdm import tqdm

# Data Preparation

In [2]:
all_subjects_all_trials_connectomes = utils.load_connectomes(utils.ALL_SUBJECT_IDS, utils.ALL_TRIAL_IDS)
all_subjects_all_trials_connectomes['fmri'].shape

(17906, 68, 68)

Extract flattened representation of upper triangular of Pearson correlation matrix for each connectome type.

In [3]:
# NOTE: The below logic would have to change if we move away from using Desikan Atlas where the number of regions 
# are the same between EEG and fMRI
num_regions = all_subjects_all_trials_connectomes['fmri'].shape[1]
num_regions

68

In [4]:
upper_triangular_including_diagonal_idxs = np.triu_indices(num_regions, k=0)
lower_triangular_idxs = np.tril_indices(num_regions, k=-1)

In [5]:
all_subjects_all_trials_connectome_upper_triangular_flattened = copy.deepcopy(all_subjects_all_trials_connectomes)
for k in all_subjects_all_trials_connectome_upper_triangular_flattened:
    all_subjects_all_trials_connectome_upper_triangular_flattened[k] = np.array([c[upper_triangular_including_diagonal_idxs].flatten() for c in all_subjects_all_trials_connectomes[k]])

In [6]:
all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'].shape

(17906, 2346)

#### Combined Modalities

In [7]:
def data_matrix_from_channels(channels):
    data_matrix = []
    for k in channels:
        data_matrix.append(all_subjects_all_trials_connectome_upper_triangular_flattened[k])
    data_matrix = np.concatenate(data_matrix, axis=1)
    return data_matrix

In [8]:
alpha_beta_delta_gamma_theta_matrix = data_matrix_from_channels(['alpha', 'beta', 'delta', 'gamma', 'theta'])
fmri_alpha_beta_delta_gamma_theta_matrix = data_matrix_from_channels(['fmri', 'alpha', 'beta', 'delta', 'gamma', 'theta'])
fmri_broad_matrix = data_matrix_from_channels(['fmri', 'broad'])

##### Train vs Test Indexes

In [9]:
N = alpha_beta_delta_gamma_theta_matrix.shape[0]
idxs = np.arange(N)
split_idx = int(0.9 * N)
train_idxs = idxs[:split_idx]
test_idxs = idxs[split_idx:]

# Hidden Markov Models

## Training

(1) TODO: Train HMMs on subset of data and predict likelihood of new data

(2) TODO: Align states between modalities.

In [10]:
def num_parameters_in_hmm(model):  
    k = model.means_.shape[0]
    d = model.means_.shape[1]
    # NOTE: We are using a diagonal covariance matrix
    return (k*d)+(k*d)+k+(k**2)

In [11]:
def train_optimal_hmm_on_data(data, bic=True, pca_variance_retained=0.99, forced_component_count=None):
    
    # Apply PCA to dimensionality reduce the data and retain xx% variance
    if pca_variance_retained < 1.0 and pca_variance_retained > 0.0:
        print("\tApplying pca and retaining {0:.2f}% variance...".format(pca_variance_retained*100))
        pca_model = PCA(pca_variance_retained)
        dim_reduced_data = pca_model.fit_transform(data)
        print("\t{0} dimensions -> {1} dimensions".format(data.shape[1], dim_reduced_data.shape[1]))
    else:
        print("Skipping dimensionality reduction...")
        pca_model = None
        dim_reduced_data = data
    
    # Compute BIC/AIC scores for HMM models with # states between 1 and N-1
    scores = []
    candidate_hmm_models = []
    
    for n_components in (range(2, data.shape[0]-1) if not forced_component_count else range(forced_component_count, forced_component_count+1)):
        
        print("\t\t training hmm with n_components={0}".format(n_components))

        # Train HMM model
        candidate_hmm_model = hmm.GaussianHMM(n_components=n_components,
                                              covariance_type="diag").fit(dim_reduced_data)
        candidate_hmm_models.append(candidate_hmm_model)

        # Compute BIC/AIC score
        n = dim_reduced_data.shape[0]
        k = num_parameters_in_hmm(candidate_hmm_model)
        L = candidate_hmm_model.decode(dim_reduced_data)[0]

        print("\t\t\tn={0}, k={1}, L={2}".format(n, k, L))

        score = (np.log(n)*k - 2*L) if bic else (k - L)
        scores.append(score)
        print("\t\t\tScore={0}".format(score))
        
        if len(scores) >= 3:
            # Last two increases in number of components yielded worse scores -> exit search
            if scores[-1] > scores[-3] and scores[-2] > scores[-3]:
                break
    
    # Select the model with the lowest BIC/AIC score
    selected_hmm_model = candidate_hmm_models[np.argmin(scores)]
    return selected_hmm_model, pca_model

In [12]:
def train_optimal_hmms_together(data_channels, bic=True, pca_variance_retained=0.99):
    
    # Apply PCA to dimensionality reduce the data and retain xx% variance
    print("\tApplying pca and retaining {0:.2f}% variance...".format(pca_variance_retained*100))
    pca_models = []
    dim_reduced_data_channels = []
    
    for data in data_channels:
        pca_model = PCA(pca_variance_retained)
        dim_reduced_data = pca_model.fit_transform(data)
        
        pca_models.append(pca_model)
        dim_reduced_data_channels.append(dim_reduced_data)
        print("\t{0} dimensions -> {1} dimensions".format(data.shape[1], dim_reduced_data.shape[1]))
    
    # Compute BIC/AIC scores for HMM models with # states between 1 and N-1
    scores = []
    candidate_hmm_models = []
    
    for n_components in range(2, len(data_channels[0])-1):
        
        print("\t\t training hmms with n_components={0}".format(n_components))

        # Train HMM model for every data channel
        candidate_hmm_models_with_n_components = []
        scores_with_n_components = []
        for dim_reduced_data in dim_reduced_data_channels:
            candidate_hmm_model = hmm.GaussianHMM(n_components=n_components,
                                                  covariance_type="diag").fit(dim_reduced_data)
            candidate_hmm_models_with_n_components.append(candidate_hmm_model)
            
            # Compute BIC/AIC score
            n = dim_reduced_data.shape[0]
            k = num_parameters_in_hmm(candidate_hmm_model)
            L = candidate_hmm_model.decode(dim_reduced_data)[0]

            print("\t\t\t\tn={0}, k={1}, L={2}".format(n, k, L))

            score = (np.log(n)*k - 2*L) if bic else (k - L)
            scores_with_n_components.append(score)
            print("\t\t\t\t\t=> Score={0}".format(score))
        
        candidate_hmm_models.append(candidate_hmm_models_with_n_components)
        scores.append(np.mean(scores_with_n_components))
        print("\t\t\tScore={0}".format(scores[-1]))


        if len(scores) >= 3:
            # Last two increases in number of components yielded worse scores -> exit search
            if scores[-1] > scores[-3] and scores[-2] > scores[-3]:
                break
                
    # Select the model with the lowest BIC/AIC score
    selected_hmm_models = candidate_hmm_models[np.argmin(scores)]
    return selected_hmm_models, pca_models

##### COMBINED EEG BANDS (Alpha+Beta+Delta+Gamma+Theta)

Full data.

In [ ]:
combined_alpha_beta_delta_gamma_theta_hmm_model = joblib.load("output/hmm/models/[hmm]-[alpha_beta_delta_gamma_theta]-[{0}].pkl".format("combined"))
combined_alpha_beta_delta_gamma_theta_pca_model = joblib.load("output/hmm/models/[pca]-[alpha_beta_delta_gamma_theta]-[{0}].pkl".format("combined"))

Training data.

In [23]:
combined_alpha_beta_delta_gamma_theta_hmm_model_train = joblib.load("output/hmm/models/[hmm]-[alpha_beta_delta_gamma_theta]-[{0}]-train.pkl".format("combined"))
combined_alpha_beta_delta_gamma_theta_pca_model_train = joblib.load("output/hmm/models/[pca]-[alpha_beta_delta_gamma_theta]-[{0}]-train.pkl".format("combined"))

##### COMBINED EEG BANDS (Broad)

Full data.

In [ ]:
broad_hmm_model = joblib.load("output/hmm/models/[hmm]-[broad].pkl")
broad_pca_model = joblib.load("output/hmm/models/[pca]-[broad].pkl")

Training data.

In [15]:
broad_hmm_model_train = joblib.load("output/hmm/models/[hmm]-[broad]-train.pkl")
broad_pca_model_train = joblib.load("output/hmm/models/[pca]-[broad]-train.pkl")

#####  COMBINED MODALITY HMM MODEL (fMRI+Alpha+Beta+Delta+Gamma+Theta)

Full data.

In [ ]:
combined_fmri_alpha_beta_delta_gamma_theta_hmm_model = joblib.load("output/hmm/models/[hmm]-[fmri_alpha_beta_delta_gamma_theta]-[{0}].pkl".format("combined"))
combined_fmri_alpha_beta_delta_gamma_theta_pca_model = joblib.load("output/hmm/models/[pca]-[fmri_alpha_beta_delta_gamma_theta]-[{0}].pkl".format("combined"))

Training data.

In [20]:
combined_fmri_alpha_beta_delta_gamma_theta_hmm_model_train = joblib.load("output/hmm/models/[hmm]-[fmri_alpha_beta_delta_gamma_theta]-[{0}]-train.pkl".format("combined"))
combined_fmri_alpha_beta_delta_gamma_theta_pca_model_train = joblib.load("output/hmm/models/[pca]-[fmri_alpha_beta_delta_gamma_theta]-[{0}]-train.pkl".format("combined"))

#####  COMBINED MODALITY HMM MODEL (fMRI+Broad)

Full data.

In [ ]:
combined_fmri_broad_hmm_model = joblib.load("output/hmm/models/[hmm]-[fmri_broad]-[{0}].pkl".format("combined"))
combined_fmri_broad_pca_model = joblib.load("output/hmm/models/[pca]-[fmri_broad]-[{0}].pkl".format("combined"))

Training data.

In [26]:
combined_fmri_broad_hmm_model_train = joblib.load("output/hmm/models/[hmm]-[fmri_broad]-[{0}]-train.pkl".format("combined"))
combined_fmri_broad_pca_model_train = joblib.load("output/hmm/models/[pca]-[fmri_broad]-[{0}]-train.pkl".format("combined"))

In [25]:
combined_fmri_broad_hmm_model_train, combined_fmri_broad_pca_model_train = train_optimal_hmm_on_data(fmri_broad_matrix[train_idxs], pca_variance_retained=0.90)
joblib.dump(combined_fmri_broad_hmm_model_train, "output/hmm/models/[hmm]-[fmri_broad]-[{0}]-train.pkl".format("combined"))
joblib.dump(combined_fmri_broad_pca_model_train, "output/hmm/models/[pca]-[fmri_broad]-[{0}]-train.pkl".format("combined"))

	Applying pca and retaining 90.00% variance...
	4692 dimensions -> 466 dimensions
		 training hmm with n_components=2
			n=16115, k=1870, L=-6714765.208506278
			Score=13447646.052847803
		 training hmm with n_components=3
			n=16115, k=2808, L=-6697689.427965185
			Score=13422581.372200621
		 training hmm with n_components=4
			n=16115, k=3748, L=-6685551.724709815
			Score=13407412.221136479
		 training hmm with n_components=5
			n=16115, k=4690, L=-6675288.471921882
			Score=13396011.346018793
		 training hmm with n_components=6
			n=16115, k=5634, L=-6663751.47386577
			Score=13382082.355376342
		 training hmm with n_components=7
			n=16115, k=6580, L=-6652728.233078584
			Score=13369200.25428333
		 training hmm with n_components=8
			n=16115, k=7528, L=-6638372.291073946
			Score=13349672.125767004
		 training hmm with n_components=9
			n=16115, k=8478, L=-6626250.173873647
			Score=13334631.02187094
		 training hmm with n_components=10
			n=16115, k=9430, L=-6615344.274538381
			

['output/hmm/models/[pca]-[fmri_broad]-[combined]-train.pkl']

##### SEPARATE EEG BANDS (Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

Full data.

In [ ]:
alpha_beta_delta_gamma_theta_hmm_models = [joblib.load("output/hmm/models/[hmm]-[alpha_beta_delta_gamma_theta]-[{0}].pkl".format(k)) for k in ['alpha', 'beta', 'delta', 'gamma', 'theta']]
alpha_beta_delta_gamma_theta_pca_models = [joblib.load("output/hmm/models/[pca]-[alpha_beta_delta_gamma_theta]-[{0}].pkl".format(k)) for k in ['alpha', 'beta', 'delta', 'gamma', 'theta']]

Training data.

In [29]:
alpha_beta_delta_gamma_theta_hmm_models_train = [joblib.load("output/hmm/models/[hmm]-[alpha_beta_delta_gamma_theta]-[{0}]-train.pkl".format(k)) for k in ['alpha', 'beta', 'delta', 'gamma', 'theta']]
alpha_beta_delta_gamma_theta_pca_models_train = [joblib.load("output/hmm/models/[pca]-[alpha_beta_delta_gamma_theta]-[{0}]-train.pkl".format(k)) for k in ['alpha', 'beta', 'delta', 'gamma', 'theta']]

##### SEPARATE MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

Full data.

In [ ]:
fmri_alpha_beta_delta_gamma_theta_pca_models = []
fmri_alpha_beta_delta_gamma_theta_hmm_models = []

for k in ['fmri', 'alpha', 'beta', 'delta', 'gamma', 'theta']:
    fmri_alpha_beta_delta_gamma_theta_pca_models.append(joblib.load("output/hmm/models/[pca]-[fmri_alpha_beta_delta_gamma_theta]-[{0}].pkl".format(k)))
    fmri_alpha_beta_delta_gamma_theta_hmm_models.append(joblib.load("output/hmm/models/[hmm]-[fmri_alpha_beta_delta_gamma_theta]-[{0}].pkl".format(k)))

Training data.

In [33]:
fmri_alpha_beta_delta_gamma_theta_pca_models_train = []
fmri_alpha_beta_delta_gamma_theta_hmm_models_train = []

for k in ['fmri', 'alpha', 'beta', 'delta', 'gamma', 'theta']:
    fmri_alpha_beta_delta_gamma_theta_pca_models_train.append(joblib.load("output/hmm/models/[pca]-[fmri_alpha_beta_delta_gamma_theta]-[{0}]-train.pkl".format(k)))
    fmri_alpha_beta_delta_gamma_theta_hmm_models_train.append(joblib.load("output/hmm/models/[hmm]-[fmri_alpha_beta_delta_gamma_theta]-[{0}]-train.pkl".format(k)))

##### SEPARATE MODALITIES (fMRI+Broad HMMs Trained Together)

Full data.

In [ ]:
fmri_broad_pca_models = []
fmri_broad_hmm_models = []

for k in ['fmri', 'broad']:
    fmri_broad_pca_models.append(joblib.load("output/hmm/models/[pca]-[fmri_broad]-[{0}].pkl".format(k)))
    fmri_broad_hmm_models.append(joblib.load("output/hmm/models/[hmm]-[fmri_broad]-[{0}].pkl".format(k)))

Training data.

In [36]:
fmri_broad_pca_models_train = []
fmri_broad_hmm_models_train = []

for k in ['fmri', 'broad']:
    fmri_broad_pca_models_train.append(joblib.load("output/hmm/models/[pca]-[fmri_broad]-[{0}]-train.pkl".format(k)))
    fmri_broad_hmm_models_train.append(joblib.load("output/hmm/models/[hmm]-[fmri_broad]-[{0}]-train.pkl".format(k)))

## Spatial Representation of Hidden States

In [ ]:
def plot_spatial_representation(hmm_models, pca_models, modalities, title, path):
    num_models_to_plot = len(hmm_models)
    num_states_to_plot = min(min([hmm_model.means_.shape[0] for hmm_model in hmm_models]), 10)
    subplot_idx = 1
    
    fig = plt.figure(figsize=(40*num_states_to_plot, 10+30*num_models_to_plot))
    fig.suptitle(title, fontsize=50)
    subplot_idx = 1
    
    for hmm_model, pca_model, modality in zip(hmm_models, pca_models, modalities):
        for state_idx in range(0, num_states_to_plot):

            # Add empty plot if necessary
            if hmm_model.means_.shape[0] <= state_idx:
                ax = fig.add_subplot(num_models_to_plot, num_states_to_plot, subplot_idx)
                subplot_idx += 1
                continue
                  
            # Extract connectome representation of the hidden state
            hidden_state = np.zeros((num_regions, num_regions))
            hidden_state[upper_triangular_including_diagonal_idxs] = pca_model.inverse_transform(hmm_model.means_[state_idx])
            hidden_state[lower_triangular_idxs] = hidden_state.T[lower_triangular_idxs]

            # Plot connectome representation of the hidden state
            ax = fig.add_subplot(num_models_to_plot, num_states_to_plot, subplot_idx)
            DesikanAtlas.plot(connectome=hidden_state,
                              title='{0} HiddenState-{1} Connectome'.format(modality, state_idx+1),
                              axes=ax)
            subplot_idx += 1

    plt.savefig(path)
    plt.close()

In [ ]:
def plot_combined_spatial_representation(combined_hmm_model, combined_pca_model, modalities, modality_feature_idxs, title, path):
    num_modalities_to_plot = len(modalities)
    num_states_to_plot = min(combined_hmm_model.means_.shape[0], 10)
    
    fig = plt.figure(figsize=(40*num_states_to_plot, 10+30*num_modalities_to_plot))
    fig.suptitle(title, fontsize=50)
    subplot_idx = 1


    for modality, feature_idxs in zip(modalities, modality_feature_idxs):
        for state_idx in range(0, num_states_to_plot):

            features = combined_pca_model.inverse_transform(combined_hmm_model.means_[state_idx])
                  
            # Extract connectome representation of the hidden state
            hidden_state = np.zeros((num_regions, num_regions))
            hidden_state[upper_triangular_including_diagonal_idxs] = features[feature_idxs]
            hidden_state[lower_triangular_idxs] = hidden_state.T[lower_triangular_idxs]

            # Plot connectome representation of the hidden state
            ax = fig.add_subplot(num_modalities_to_plot, num_states_to_plot, subplot_idx)
            DesikanAtlas.plot(connectome=hidden_state,
                              title='{0} HiddenState-{1} Connectome'.format(modality, state_idx+1),
                              axes=ax)
            subplot_idx += 1

    plt.savefig(path)
    plt.close()

##### COMBINED EEG BANDS (Alpha+Beta+Delta+Gamma+Theta)

In [ ]:
plot_combined_spatial_representation(combined_hmm_model=combined_alpha_beta_delta_gamma_theta_hmm_model,
                                    combined_pca_model=combined_alpha_beta_delta_gamma_theta_pca_model,
                                    modalities=["Alpha", "Beta", "Delta", "Gamma", "Theta"],
                                    modality_feature_idxs=[
                                        range(2346*0, 2346*1),
                                        range(2346*1, 2346*2),
                                        range(2346*2, 2346*3),
                                        range(2346*3, 2346*4),
                                        range(2346*4, 2346*5),
                                    ],
                                    title="Connectome Representation of States of Combined Alpha+Beta+Delta+Gamma+Theta Hidden Markov Model",
                                    path="output/hmm/spatial_representation-[alpha_beta_delta_gamma_theta]-[combined].png")

##### COMBINED EEG BANDS (Broad)

In [ ]:
plot_combined_spatial_representation(combined_hmm_model=broad_hmm_model,
                                    combined_pca_model=broad_pca_model,
                                    modalities=["Broad"],
                                    modality_feature_idxs=[
                                        range(2346*0, 2346*1),
                                    ],
                                    title="Connectome Representation of States of Broad Hidden Markov Model",
                                    path="output/hmm/spatial_representation-[broad]-[combined].png")

#####  COMBINED MODALITY HMM MODEL (fMRI+Alpha+Beta+Delta+Gamma+Theta)

In [ ]:
plot_combined_spatial_representation(combined_hmm_model=combined_fmri_alpha_beta_delta_gamma_theta_hmm_model,
                                    combined_pca_model=combined_fmri_alpha_beta_delta_gamma_theta_pca_model,
                                    modalities=["fMRI", "Alpha", "Beta", "Delta", "Gamma", "Theta"],
                                    modality_feature_idxs=[
                                        range(2346*0, 2346*1),
                                        range(2346*1, 2346*2),
                                        range(2346*2, 2346*3),
                                        range(2346*3, 2346*4),
                                        range(2346*4, 2346*5),
                                        range(2346*5, 2346*6),
                                    ],
                                    title="Connectome Representation of States of Combined fMRI+Alpha+Beta+Delta+Gamma+Theta Hidden Markov Model",
                                    path="output/hmm/spatial_representation-[fmri_alpha_beta_delta_gamma_theta]-[combined].png")

#####  COMBINED MODALITY HMM MODEL (fMRI+Broad)

In [ ]:
plot_combined_spatial_representation(combined_hmm_model=combined_fmri_broad_hmm_model,
                                    combined_pca_model=combined_fmri_broad_pca_model,
                                    modalities=["fMRI", "Broad"],
                                    modality_feature_idxs=[
                                        range(2346*0, 2346*1),
                                        range(2346*1, 2346*2),
                                    ],
                                    title="Connectome Representation of States of Combined fMRI+Broad Hidden Markov Model",
                                    path="output/hmm/spatial_representation-[fmri_broad]-[combined].png")

##### SEPARATE EEG BANDS (Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

In [ ]:
plot_spatial_representation(hmm_models=alpha_beta_delta_gamma_theta_hmm_models,
                            pca_models=alpha_beta_delta_gamma_theta_pca_models,
                            modalities=["Alpha", "Beta", "Delta", "Gamma", "Theta"],
                            title="Connectome Representation of Jointly Trained Alpha+Beta+Delta+Gamma+Theta Hidden Markov Model States",
                            path="output/hmm/spatial_representation-[alpha_beta_delta_gamma_theta]-[separate].png")

##### SEPARATE MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

In [ ]:
plot_spatial_representation(hmm_models=fmri_alpha_beta_delta_gamma_theta_hmm_models,
                            pca_models=fmri_alpha_beta_delta_gamma_theta_pca_models,
                            modalities=["fMRI", "Alpha", "Beta", "Delta", "Gamma", "Theta"],
                            title="Connectome Representation of Jointly Trained fMRI+Alpha+Beta+Delta+Gamma+Theta Hidden Markov Model States",
                            path="output/hmm/spatial_representation-[fmri_alpha_beta_delta_gamma_theta]-[separate].png")

##### SEPARATE MODALITIES (fMRI+Broad HMMs Trained Together)

In [ ]:
plot_spatial_representation(hmm_models=fmri_broad_hmm_models,
                            pca_models=fmri_broad_pca_models,
                            modalities=["fMRI", "Broad"],
                            title="Connectome Representation of Jointly Trained fMRI+Broad Hidden Markov Model States",
                            path="output/hmm/spatial_representation-[fmri_broad]-[separate].png")

## Hidden State Sequence Likelihood Comparison

##### COMBINED EEG BANDS (Alpha+Beta+Delta+Gamma+Theta)

Full data.

In [ ]:
combined_alpha_beta_delta_gamma_theta_hmm_model.means_.shape[0]

In [ ]:
combined_alpha_beta_delta_gamma_theta_log_likelihood, combined_alpha_beta_delta_gamma_theta_decoded_state_sequence = combined_alpha_beta_delta_gamma_theta_hmm_model.decode(combined_alpha_beta_delta_gamma_theta_pca_model.transform(alpha_beta_delta_gamma_theta_matrix))
combined_alpha_beta_delta_gamma_theta_log_likelihood

Training data.

In [37]:
combined_alpha_beta_delta_gamma_theta_hmm_model_train.means_.shape[0]

4

In [38]:
combined_alpha_beta_delta_gamma_theta_log_likelihood_test, _ = combined_alpha_beta_delta_gamma_theta_hmm_model_train.decode(combined_alpha_beta_delta_gamma_theta_pca_model_train.transform(alpha_beta_delta_gamma_theta_matrix[test_idxs]))
combined_alpha_beta_delta_gamma_theta_log_likelihood_test

8098097.956017438

##### COMBINED EEG BANDS (Broad)

Full data.

In [ ]:
broad_hmm_model.means_.shape[0]

In [ ]:
broad_log_likelihood, broad_decoded_state_sequence = broad_hmm_model.decode(broad_pca_model.transform(all_subjects_all_trials_connectome_upper_triangular_flattened['broad']))
broad_log_likelihood

Training data.

In [39]:
broad_hmm_model_train.means_.shape[0]

5

In [40]:
broad_log_likelihood_test, _ = broad_hmm_model_train.decode(broad_pca_model_train.transform(all_subjects_all_trials_connectome_upper_triangular_flattened['broad'][test_idxs]))
broad_log_likelihood_test

4123704.3675378673

##### COMBINED MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta)

Full data.

In [ ]:
combined_fmri_alpha_beta_delta_gamma_theta_hmm_model.means_.shape[0]

In [ ]:
combined_fmri_alpha_beta_delta_gamma_theta_log_likelihood, combined_fmri_alpha_beta_delta_gamma_theta_decoded_state_sequence = combined_fmri_alpha_beta_delta_gamma_theta_hmm_model.decode(combined_fmri_alpha_beta_delta_gamma_theta_pca_model.transform(fmri_alpha_beta_delta_gamma_theta_matrix))
combined_fmri_alpha_beta_delta_gamma_theta_log_likelihood

Training data.

In [41]:
combined_fmri_alpha_beta_delta_gamma_theta_hmm_model_train.means_.shape[0]

6

In [42]:
combined_fmri_alpha_beta_delta_gamma_theta_log_likelihood_test, _ = combined_fmri_alpha_beta_delta_gamma_theta_hmm_model_train.decode(combined_fmri_alpha_beta_delta_gamma_theta_pca_model_train.transform(fmri_alpha_beta_delta_gamma_theta_matrix[test_idxs]))
combined_fmri_alpha_beta_delta_gamma_theta_log_likelihood_test

-229639.5274247451

#####  COMBINED MODALITIES (fMRI+Broad)

Full data.

In [ ]:
combined_fmri_broad_hmm_model.means_.shape[0]

In [ ]:
combined_fmri_broad_log_likelihood, combined_fmri_broad_decoded_state_sequence = combined_fmri_broad_hmm_model.decode(combined_fmri_broad_pca_model.transform(fmri_broad_matrix))
combined_fmri_broad_log_likelihood

Training data.

In [43]:
combined_fmri_broad_hmm_model_train.means_.shape[0]

51

In [44]:
combined_fmri_broad_log_likelihood_test, _ = combined_fmri_broad_hmm_model_train.decode(combined_fmri_broad_pca_model_train.transform(fmri_broad_matrix[test_idxs]))
combined_fmri_broad_log_likelihood_test

-668462.1159546989

##### SEPARATE EEG BANDS (Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

Full data.

In [ ]:
alpha_beta_delta_gamma_theta_hmm_models[0].means_.shape[0]

In [ ]:
alpha_beta_delta_gamma_theta_decoded_state_sequences = []
alpha_beta_delta_gamma_theta_decoded_log_likelihoods = []
alpha_beta_delta_gamma_theta_data_channels = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['alpha'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['beta'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['delta'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['gamma'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['theta']
]
for (data_channel, hmm_model, pca_model) in zip(alpha_beta_delta_gamma_theta_data_channels, alpha_beta_delta_gamma_theta_hmm_models, alpha_beta_delta_gamma_theta_pca_models):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    alpha_beta_delta_gamma_theta_decoded_log_likelihoods.append(decoded_state_sequence[0])
    alpha_beta_delta_gamma_theta_decoded_state_sequences.append(decoded_state_sequence[1])

In [ ]:
print(alpha_beta_delta_gamma_theta_decoded_log_likelihoods)
print(np.mean(alpha_beta_delta_gamma_theta_decoded_log_likelihoods))

Training data.

In [45]:
alpha_beta_delta_gamma_theta_hmm_models_train[0].means_.shape[0]

6

In [46]:
alpha_beta_delta_gamma_theta_decoded_log_likelihoods_test = []
alpha_beta_delta_gamma_theta_data_channels_test = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['alpha'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['beta'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['delta'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['gamma'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['theta'][test_idxs],
]
for (data_channel, hmm_model, pca_model) in zip(alpha_beta_delta_gamma_theta_data_channels_test, alpha_beta_delta_gamma_theta_hmm_models_train, alpha_beta_delta_gamma_theta_pca_models_train):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    alpha_beta_delta_gamma_theta_decoded_log_likelihoods_test.append(decoded_state_sequence[0])

In [47]:
print(alpha_beta_delta_gamma_theta_decoded_log_likelihoods_test)
print(np.mean(alpha_beta_delta_gamma_theta_decoded_log_likelihoods_test))

[2425505.8387772166, 3712908.4900000384, 2404498.679815574, 3210840.040412554, 2448858.8345558573]
2840522.376712248


##### SEPARATE MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

Full data.

In [ ]:
fmri_alpha_beta_delta_gamma_theta_hmm_models[0].means_.shape[0]

In [ ]:
fmri_alpha_beta_delta_gamma_theta_decoded_state_sequences = []
fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods = []
fmri_alpha_beta_delta_gamma_theta_data_channels = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['alpha'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['beta'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['delta'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['gamma'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['theta']
]
for (data_channel, hmm_model, pca_model) in zip(fmri_alpha_beta_delta_gamma_theta_data_channels, fmri_alpha_beta_delta_gamma_theta_hmm_models, fmri_alpha_beta_delta_gamma_theta_pca_models):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods.append(decoded_state_sequence[0])
    fmri_alpha_beta_delta_gamma_theta_decoded_state_sequences.append(decoded_state_sequence[1])

In [ ]:
print(fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods)
print(np.mean(fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods))

Training data.

In [48]:
fmri_alpha_beta_delta_gamma_theta_hmm_models_train[0].means_.shape[0]

6

In [49]:
fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods_test = []
fmri_alpha_beta_delta_gamma_theta_data_channels_test = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['alpha'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['beta'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['delta'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['gamma'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['theta'][test_idxs],
]
for (data_channel, hmm_model, pca_model) in zip(fmri_alpha_beta_delta_gamma_theta_data_channels_test, fmri_alpha_beta_delta_gamma_theta_hmm_models_train, fmri_alpha_beta_delta_gamma_theta_pca_models_train):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods_test.append(decoded_state_sequence[0])

In [50]:
print(fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods_test)
print(np.mean(fmri_alpha_beta_delta_gamma_theta_decoded_log_likelihoods_test))

[-632060.8769609219, 2425507.8935020478, 3713007.0461099097, 2404024.220427762, 3211548.8042127336, 2449031.872246883]
2261843.159923069


##### SEPARATE MODALITIES (fMRI+Broad HMMs Trained Together)

Full data.

In [ ]:
fmri_broad_hmm_models[0].means_.shape[0]

In [ ]:
fmri_broad_decoded_state_sequences = []
fmri_broad_decoded_log_likelihoods = []
fmri_broad_data_channels = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'],
    all_subjects_all_trials_connectome_upper_triangular_flattened['broad'],
]
for (data_channel, hmm_model, pca_model) in zip(fmri_broad_data_channels, fmri_broad_hmm_models, fmri_broad_pca_models):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    fmri_broad_decoded_log_likelihoods.append(decoded_state_sequence[0])
    fmri_broad_decoded_state_sequences.append(decoded_state_sequence[1])

In [ ]:
print(fmri_broad_decoded_log_likelihoods)
print(np.mean(fmri_broad_decoded_log_likelihoods))

Training data.

In [51]:
fmri_broad_hmm_models_train[0].means_.shape[0]

7

In [52]:
fmri_broad_decoded_log_likelihoods_test = []
fmri_broad_data_channels_test = [
    all_subjects_all_trials_connectome_upper_triangular_flattened['fmri'][test_idxs],
    all_subjects_all_trials_connectome_upper_triangular_flattened['broad'][test_idxs],
]
for (data_channel, hmm_model, pca_model) in zip(fmri_broad_data_channels_test, fmri_broad_hmm_models_train, fmri_broad_pca_models_train):
    dim_reduced_data = pca_model.transform(data_channel)
    decoded_state_sequence = hmm_model.decode(dim_reduced_data)
    fmri_broad_decoded_log_likelihoods_test.append(decoded_state_sequence[0])

In [53]:
print(fmri_broad_decoded_log_likelihoods_test)
print(np.mean(fmri_broad_decoded_log_likelihoods_test))

[-633789.2878622932, 4124414.754839838]
1745312.7334887725


## Hidden State Sequence Statistics

In [ ]:
def get_n_colors(n):
        return [ cmx.rainbow(float(i)/n) for i in range(n) ]

In [ ]:
def plot_hidden_state_sequence_statistics(hmm_model, hidden_state_sequence, title, path):
    
    num_statistics_to_plot = 4
    num_plots_per_statistic = 1
    subplot_idx = 1
    
    fig = plt.figure(figsize=(100*num_plots_per_statistic, 20))
    fig.suptitle(title, fontsize=50)
    subplot_idx = 1
    
    hidden_state_colors = get_n_colors(hmm_model.means_.shape[0])

    # Timeseries Sample
    ax = plt.subplot2grid((3, 3), (0, 0), colspan=3)
    x = 0
    for state in hidden_state_sequence[:1000]:
        ax.axvline(x=x, color=hidden_state_colors[state])
        x += 1
    plt.title("Hidden State Sequence")
    plt.yticks([])
    subplot_idx += 1
    
    # Fractional Occupancy
    fractional_occupancies_per_state = []
    for state_idx in range(0, hmm_model.means_.shape[0]):
        fractional_occupancy = len(hidden_state_sequence[hidden_state_sequence == state_idx])/len(hidden_state_sequence)
        fractional_occupancies_per_state.append(fractional_occupancy)

    ax = plt.subplot2grid((3, 3), (2, 0))
    plt.title("Fractional Occupancy")
    x = np.arange(hmm_model.means_.shape[0])
    plt.bar(x, height=fractional_occupancies_per_state)
    plt.xticks(x, [str(x_i) for x_i in x])
    subplot_idx += 1
    
    # Mean Life Time
    mean_life_time_per_state = []
    for state_idx in range(0, hmm_model.means_.shape[0]):
        
        # Count number of transitions out of state with state_idx
        num_transitions_out_of_state_with_state_idx = 0
        for i in range(0, len(hidden_state_sequence)-1):
            if hidden_state_sequence[i] == state_idx and hidden_state_sequence[i+1] != state_idx:
                num_transitions_out_of_state_with_state_idx += 1
        
        # Count total number of time points spent in state with state idx
        num_time_points_in_state_with_state_idx = len(hidden_state_sequence[hidden_state_sequence == state_idx])
        
        # Compute mean life time
        mean_life_time = num_time_points_in_state_with_state_idx/num_transitions_out_of_state_with_state_idx
        mean_life_time_per_state.append(mean_life_time)
    
    
    ax = plt.subplot2grid((3, 3), (2, 1))
    plt.title('Mean Life Time')
    plt.bar(x, height=mean_life_time_per_state)
    plt.xticks(x, [str(x_i) for x_i in x])
    subplot_idx += 1
    
    # Transition Probabilities
    ax = plt.subplot2grid((3, 3), (2, 2))
    plt.imshow(hmm_model.transmat_, cmap='gist_heat')
    plt.title("Transition Probabilities")
    plt.colorbar()
    subplot_idx += 1

    plt.savefig(path)
    plt.close()

##### COMBINED EEG BANDS (Alpha+Beta+Delta+Gamma+Theta)

In [ ]:
plot_hidden_state_sequence_statistics(hmm_model=combined_alpha_beta_delta_gamma_theta_hmm_model,
                                      hidden_state_sequence=combined_alpha_beta_delta_gamma_theta_decoded_state_sequence,
                                      title="Hidden State Sequence Statistics of Combined Alpha+Beta+Delta+Gamma+Theta HMM",
                                      path="output/hmm/hidden-state-sequence-statistics-[alpha_beta_delta_gamma_theta]-[combined].png")

##### COMBINED EEG BANDS (Broad)

In [ ]:
plot_hidden_state_sequence_statistics(hmm_model=broad_hmm_model,
                                      hidden_state_sequence=broad_decoded_state_sequence,
                                      title="Hidden State Sequence Statistics of Combined Broad HMM",
                                      path="output/hmm/hidden-state-sequence-statistics-[broad]-[combined].png")

##### COMBINED MODALITY HMM MODEL (fMRI+Alpha+Beta+Delta+Gamma+Theta)

In [ ]:
plot_hidden_state_sequence_statistics(hmm_model=combined_fmri_alpha_beta_delta_gamma_theta_hmm_model,
                                      hidden_state_sequence=combined_fmri_alpha_beta_delta_gamma_theta_decoded_state_sequence,
                                      title="Hidden State Sequence Statistics of Combined fMRI+Alpha+Beta+Delta+Gamma+Theta HMM",
                                      path="output/hmm/hidden-state-sequence-statistics-[fmri_alpha_beta_delta_gamma_theta]-[combined].png")

##### COMBINED MODALITY HMM MODEL (fMRI+Broad)

In [ ]:
plot_hidden_state_sequence_statistics(hmm_model=combined_fmri_broad_hmm_model,
                                      hidden_state_sequence=combined_fmri_broad_decoded_state_sequence,
                                      title="Hidden State Sequence Statistics of Combined fMRI+Broad HMM",
                                      path="output/hmm/hidden-state-sequence-statistics-[fmri_broad]-[combined].png")

# Brain Graph Statistics

Plot timeseries of graph statistics.

Compute correlation between all possible pairs of statistics across modalities.

In [ ]:
brain_graph_statistics_descriptions = [
    "fMRI Modularity",
    "Alpha-Band EEG Modularity",
    "Beta-Band EEG Modularity",
    "Delta-Band EEG Modularity",
    "Gamma-Band EEG Modularity",
    "Theta-Band EEG Modularity",
    
    "fMRI Assortativity",
    "Alpha-Band EEG Assortativity",
    "Beta-Band EEG Assortativity",
    "Delta-Band EEG Assortativity",
    "Gamma-Band EEG Assortativity",
    "Theta-Band EEG Assortativity",
    
    "fMRI Global Efficiency",
    "Alpha-Band EEG Global Efficiency",
    "Beta-Band EEG Global Efficiency",
    "Delta-Band EEG Global Efficiency",
    "Gamma-Band EEG Global Efficiency",
    "Theta-Band EEG Global Efficiency",
]

In [ ]:
brain_graph_statistics = pickle.load(open('output/hmm/graph_statistics/brain_graph_statistics_time_series.pkl', 'rb'))
brain_graph_statistics_means = pickle.load(open('output/hmm/graph_statistics/brain_graph_statistics_means.pkl', 'rb'))
brain_graph_statistics_vars = pickle.load(open('output/hmm/graph_statistics/brain_graph_statistics_vars.pkl', 'rb'))

In [ ]:
brain_graph_statistics.shape

### HMM Hidden State Analysis Through Brain Graph Statistics

In [ ]:
def plot_combined_hidden_state_statistics(hmm_model, hmm_model_decoded_state_sequence, title, path):
    num_states_to_plot = min(hmm_model.means_.shape[0], 10)
    subplot_idx = 1
    
    fig = plt.figure(figsize=(20*num_states_to_plot, 20))
    fig.suptitle(title, fontsize=50)
    subplot_idx = 1
    
    for state_idx in range(0, num_states_to_plot):
        
        time_series_idxs_in_state = hmm_model_decoded_state_sequence[hmm_model_decoded_state_sequence == state_idx]
        time_series_statistics_for_state = brain_graph_statistics[time_series_idxs_in_state]
        hidden_state_statistics = np.mean(time_series_statistics_for_state, axis=0)
        
        fig.add_subplot(1, num_states_to_plot, subplot_idx)
        x = 0
        for stat, stat_mean, stat_var in zip(hidden_state_statistics, brain_graph_statistics_means, brain_graph_statistics_vars):
            z_stat = (stat - stat_mean)/np.sqrt(stat_var)
            plt.bar(x, z_stat, align='center', width=0.7, color='blue' if z_stat > 0 else 'red')
            x += 1
        plt.ylim([-1.96, 1.96])
        plt.ylabel("Z-Score of Statistic in Hidden State Compared to Overall Timeseries")
        plt.title("Hidden State {0}".format(state_idx))
        plt.xticks(range(len(brain_graph_statistics_descriptions)), brain_graph_statistics_descriptions, rotation='vertical')
        subplot_idx += 1
        

    plt.savefig(path)
    plt.close()

In [ ]:
def plot_hidden_state_statistics(hmm_models, hmm_model_decoded_state_sequences, modalities, title, path):
    num_models_to_plot = len(hmm_models)
    num_states_to_plot = min(min([hmm_model.means_.shape[0] for hmm_model in hmm_models]), 10)
    subplot_idx = 1
    
    fig = plt.figure(figsize=(20*num_states_to_plot, 20+20*num_models_to_plot))
    fig.suptitle(title, fontsize=50)
    subplot_idx = 1
    
    for hmm_model, hmm_model_decoded_state_sequence, modality in zip(hmm_models, hmm_model_decoded_state_sequences, modalities):
        for state_idx in range(0, num_states_to_plot):

            time_series_idxs_in_state = hmm_model_decoded_state_sequence[hmm_model_decoded_state_sequence == state_idx]
            time_series_statistics_for_state = brain_graph_statistics[time_series_idxs_in_state]
            hidden_state_statistics = np.mean(time_series_statistics_for_state, axis=0)

            fig.add_subplot(num_models_to_plot, num_states_to_plot, subplot_idx)
            x = 0
            for stat, stat_mean, stat_var in zip(hidden_state_statistics, brain_graph_statistics_means, brain_graph_statistics_vars):
                z_stat = (stat - stat_mean)/np.sqrt(stat_var)
                plt.bar(x, z_stat, align='center', width=0.7, color='blue' if z_stat > 0 else 'red')
                x += 1
            plt.ylim([-1.96, 1.96])
            plt.ylabel("Z-Score of Statistic in Hidden State Compared to Overall Timeseries")
            plt.title("{0} Hidden State {1}".format(modality, state_idx))
            plt.xticks(range(len(brain_graph_statistics_descriptions)), brain_graph_statistics_descriptions, rotation='vertical')
            subplot_idx += 1
        
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

##### COMBINED EEG BANDS (Alpha+Beta+Delta+Gamma+Theta)

In [ ]:
plot_combined_hidden_state_statistics(hmm_model=combined_alpha_beta_delta_gamma_theta_hmm_model,
                                     hmm_model_decoded_state_sequence=combined_alpha_beta_delta_gamma_theta_decoded_state_sequence,
                                     title="Brain Graph Statistic Hidden State Analysis of Combined Alpha+Beta+Delta+Gamma+Theta HMM",
                                     path='output/hmm/hidden-state-statistics-[alpha_beta_delta_gamma_theta]-[combined].png')

##### COMBINED EEG BANDS (Broad)

In [ ]:
plot_combined_hidden_state_statistics(hmm_model=broad_hmm_model,
                                     hmm_model_decoded_state_sequence=broad_decoded_state_sequence,
                                     title="Brain Graph Statistic Hidden State Analysis of Broad HMM",
                                     path='output/hmm/hidden-state-statistics-[broad]-[combined].png')

##### COMBINED MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta)

In [ ]:
plot_combined_hidden_state_statistics(hmm_model=combined_fmri_alpha_beta_delta_gamma_theta_hmm_model,
                                     hmm_model_decoded_state_sequence=combined_fmri_alpha_beta_delta_gamma_theta_decoded_state_sequence,
                                     title="Brain Graph Statistic Hidden State Analysis of fMRI+Alpha+Beta+Delta+Gamma+Theta HMM",
                                     path='output/hmm/hidden-state-statistics-[fmri_alpha_beta_delta_gamma_theta]-[combined].png')

##### COMBINED MODALITIES (fMRI+Broad)

In [ ]:
plot_combined_hidden_state_statistics(hmm_model=combined_fmri_broad_hmm_model,
                                     hmm_model_decoded_state_sequence=combined_fmri_broad_decoded_state_sequence,
                                     title="Brain Graph Statistic Hidden State Analysis of fMRI+Broad HMM",
                                     path='output/hmm/hidden-state-statistics-[fmri_broad]-[combined].png')

##### SEPARATE EEG BANDS (Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

In [ ]:
plot_hidden_state_statistics(hmm_models=alpha_beta_delta_gamma_theta_hmm_models,
                            hmm_model_decoded_state_sequences=alpha_beta_delta_gamma_theta_decoded_state_sequences,
                            modalities=['Alpha', 'Beta', 'Delta', 'Gamma', 'Theta'],
                            title="Brain Graph Statistic Hidden State Analysis of Jointly Trained Alpha+Beta+Delta+Gamma+Theta HMMs",
                            path='output/hmm/hidden-state-statistics-[alpha_beta_delta_gamma_theta]-[separate].png')

##### SEPARATE MODALITIES (fMRI+Alpha+Beta+Delta+Gamma+Theta HMMs Trained Together)

In [ ]:
plot_hidden_state_statistics(hmm_models=fmri_alpha_beta_delta_gamma_theta_hmm_models,
                            hmm_model_decoded_state_sequences=fmri_alpha_beta_delta_gamma_theta_decoded_state_sequences,
                            modalities=['fMRI', 'Alpha', 'Beta', 'Delta', 'Gamma', 'Theta'],
                            title="Brain Graph Statistic Hidden State Analysis of Jointly Trained fMRI+Alpha+Beta+Delta+Gamma+Theta HMMs",
                            path='output/hmm/hidden-state-statistics-[fmri_alpha_beta_delta_gamma_theta]-[separate].png')

##### SEPARATE MODALITIES (fMRI+Broad HMMs Trained Together)

In [ ]:
plot_hidden_state_statistics(hmm_models=fmri_broad_hmm_models,
                            hmm_model_decoded_state_sequences=fmri_broad_decoded_state_sequences,
                            modalities=['fMRI', 'Broad'],
                            title="Brain Graph Statistic Hidden State Analysis of Jointly Trained fMRI+Broad HMMs",
                            path='output/hmm/hidden-state-statistics-[fmri_broad]-[separate].png')

##### Fit a HMM to the time series of the brain graph statistics

In [ ]:
brain_graph_statistics_hmm_model, _ = train_optimal_hmm_on_data(brain_graph_statistics, pca_variance_retained=1.0)

In [ ]:
brain_graph_statistics_hmm_model.means_.shape

In [ ]:
# Hidden State Analysis
num_hidden_states = brain_graph_statistics_hmm_model.means_.shape[0]

f = plt.figure(figsize=(10*num_hidden_states, 20))
f.suptitle('Brain Graph Statistic HMM Hidden State Analysis')
subplot_idx = 1

for i in range(num_hidden_states):

    hidden_state = brain_graph_statistics_hmm_model.means_[i]

    f.add_subplot(1, num_hidden_states, subplot_idx)
    x = 0
    for stat, stat_mean, stat_var in zip(hidden_state, brain_graph_statistics_means, brain_graph_statistics_vars):
        z_stat = (stat - stat_mean)/np.sqrt(stat_var)
        plt.bar(x, z_stat, align='center', width=0.7, color='blue' if z_stat > 0 else 'red')
        x += 1
    plt.ylim([-1.96, 1.96])
    plt.ylabel("Z-Score of Statistic in Hidden State Compared to Overall Timeseries")
    plt.title("Hidden State {0}".format(i))
    plt.xticks(range(len(brain_graph_statistics_descriptions)), brain_graph_statistics_descriptions, rotation='vertical')
    subplot_idx += 1

plt.subplots_adjust(hspace=0.5)
f.savefig('output/hmm/graph_statistic_hmm_hidden_state_analysis.png')
plt.close()

In [ ]:
# Transition Probabilities    
plt.imshow(brain_graph_statistics_hmm_model.transmat_, cmap='gist_heat')
plt.title('Brain Graph Statistic HMM Transition Probabilities')
plt.xticks(np.arange(0, brain_graph_statistics_hmm_model.transmat_.shape[0], 1))
plt.yticks(np.arange(0, brain_graph_statistics_hmm_model.transmat_.shape[0], 1))
plt.colorbar()
plt.savefig('output/hmm/graph_statistic_transition_probabilities.png')
plt.close()

In [ ]:
# Decoded State Sequence
brain_graph_statistics_hmm_decoded_log_likelihood, brain_graph_statistics_hmm_decoded_state_sequence = brain_graph_statistics_hmm_model.decode(brain_graph_statistics)

In [ ]:
# Mean Life Time Per State
mean_life_time_per_state = []
for state_id in range(0, brain_graph_statistics_hmm_model.means_.shape[0]):

    # Count number of transitions out of state with state_id
    num_transitions_out_of_state_with_state_id = 0
    for i in range(0, len(brain_graph_statistics_hmm_decoded_state_sequence)-1):
        if brain_graph_statistics_hmm_decoded_state_sequence[i] == state_id and brain_graph_statistics_hmm_decoded_state_sequence[i+1] != state_id:
            num_transitions_out_of_state_with_state_id += 1

    # Count total number of time points spent in state with state id
    num_time_points_in_state_with_state_id = len(brain_graph_statistics_hmm_decoded_state_sequence[brain_graph_statistics_hmm_decoded_state_sequence == state_id])

    # Compute mean life time
    mean_life_time = num_time_points_in_state_with_state_id/num_transitions_out_of_state_with_state_id
    mean_life_time_per_state.append(mean_life_time)


# Plot mean life time per state
plt.title('Brain Graph Statistic HMM Hidden State Mean Life Time')
x = np.arange(brain_graph_statistics_hmm_model.means_.shape[0])
plt.bar(x, height=mean_life_time_per_state)
plt.xticks(x, [str(x_i) for x_i in x])

plt.savefig('output/hmm/graph_statistic_hmm_mean_life_time.png')
plt.close()

In [ ]:
# Fractional Occupancy Per State
fractional_occupancies_per_state = []
for state_idx in range(0, brain_graph_statistics_hmm_model.means_.shape[0]):
    fractional_occupancy = len(brain_graph_statistics_hmm_decoded_state_sequence[brain_graph_statistics_hmm_decoded_state_sequence == state_idx])/len(brain_graph_statistics_hmm_decoded_state_sequence)
    fractional_occupancies_per_state.append(fractional_occupancy)

# Plot fractional occupancy per state
plt.title('Brain Graph Statistic HMM Hidden State Fractional Occupancy')
x = np.arange(brain_graph_statistics_hmm_model.means_.shape[0])
plt.bar(x, height=fractional_occupancies_per_state)
plt.xticks(x, [str(x_i) for x_i in x])
plt.savefig('output/hmm/graph_statistic_hmm_fractional_occupancy.png')
plt.close()

In [ ]:
TODO: Compute Controllability Statistic